Capstone Project - The Battle of Neighborhoods

Introduction

Evey time I go abroad I have the same problem of which hotel to choose and which nice places there are around the hotel. I usually short of time and I need everything to be close to me in the best way possibe. I want to find the best hotels in the best locations. Everywhere.

Business Problem

The searching procedure of hotels can be very frustrating. You don't realy know the city and where is the best places for you to stay depending on the quality of the hotel and the surroundings. It can take a lot of time to find a hotel which provides you the best deal for your stay. Everyone who is going to stay in a hotelcan be a potentional client of this project.

Strategy

The aim of my work is to use machine learning in order to find the best hotels in the best areas and let the costumer to choose from a very limited collection of "the best of the best" what hotel is his number 1 hotel. This work will Use Foursquare API to get the Data of Hotels in any City provided by the User, it will display ratings of the hotels. So we can Rank Hotels. the code will also search for Nearby Places for the Top 5 Hotels and Display them to the user. It will help them to choosing the Hotel based on their Personal Favorable Nearby Places.

Data

User will write down the name of the city and using Geopy Library, he will get it's coordinates. Using Folium library, a map of his chosen location will be made. On top of that map he will see the hotels and the surroundings.
Using Foursquare API the user will get hotels in a redius of 7 KM from his city of choice. A JSON file will be produced and it will be converted to Pandas DataFrame containing all the relevant hotels data.
Having hotel ID's, using foursquare API, the user will get ratings of the hotels in a data frame.
The next stage is to get top 5 by sorting them into a data frame.
Having the hotels locations, Foursquare API can find places in the area around the hotel. This data frame will be combined with the hotels data frame.
Finally, a map all Nearby Places along with the 5 Hotels will be generated with clusters of Hotels + Nearby Places using K-Means Algorithm.

Import the relevant libraries:

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')



1. Check for hotels info at Foursquare


In [ ]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'W1LVTWIPDLTONF0IUDBTE3ZW2KAG4NXB4MAXHLPWVFRXBJXZ' # your Foursquare ID
CLIENT_SECRET = '0FTT5AYQ3PH0N50PTR0IAUYZMIR35UVTDE5A1SK1XYT1H3GU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [ ]:
# Search for hotels in Miami
search_query = 'hotel'
radius = 70000 # 7 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_hotels = requests.get(url).json()
result_hotels

In [ ]:
#create a map of Miami area:
Miami = folium.Map(location=[latitude, longitude], zoom_start=12)
   
Miami

Arrange the data:

In [ ]:

# assign relevant part of JSON to hotel
hotels = result_hotels['response']['venues']

# tranform hotels into a dataframe
df_hotels = json_normalize(hotels)
df_hotels.shape

In [ ]:
# keep only columns that include hotel name, and anything that is associated with location:
filtered_columns = ['name', 'categories'] + [col for col in df_hotels.columns if col.startswith('location.')] + ['id']
df_hotels_filtered = df_hotels.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotels_filtered['categories'] = df_hotels_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotels_filtered.columns = [column.split('.')[-1] for column in df_hotels_filtered.columns]

pd.DataFrame(df_hotels_filtered)
df_hotels_filtered

In [ ]:

# drop unrelevant columns:
df_hotels_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)
df_hotels_filtered.head()

In [ ]:
# create map of Miami using latitude and longitude values:
map_miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_hotels_filtered['lat'], df_hotels_filtered['lng'], df_hotels_filtered['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  
    
map_miami

Methodology

Our aim is to search for the best hotels in a 7 KM Radius from Miami coordinates
We have collected the required data: location(coordinates) and type (category) of every Hotel (From Foursquare API).
Next, using foursquare API, we will gather hotels rating and focus on top 5 and their surroundings.
In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: We will Cluster Hotel and their Nearby Places using K-means Clustering.

2. Get Ratings for All Hotels

Using For Loop we will Iterate to every Hotels and Fetch Corresponding Ratings and then Store it in new dataframe ratings

In [ ]:

ratings = pd.DataFrame()
list1 = pd.Series([]) 
list2 = pd.Series([]) 
for i in range(len(df_hotels_filtered.id)):
    hotel_id = df_hotels_filtered.id[i] # ID of Hotels
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(hotel_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #Let's Check for Ratings, If Available then store it and if not, then store 0
    try:
        list1[i] = result['response']['venue']['rating'] 
        list2[i] = df_hotels_filtered.id[i]
    except:
        list1[i] = 0
        list2[i] = df_hotels_filtered.id[i]
         
        
ratings.insert(0, "ID", list2, allow_duplicates=True)
ratings.insert(1, "Ratings", list1, allow_duplicates=True)
ratings

In [ ]:

#merge both dataframes based on id
ratings.columns = ['id', 'Ratings']
df_hotels_filtered = pd.merge(df_hotels_filtered,
                 ratings,
                 on='id')
df_hotels_filtered

Lets Remove Hotels with No Ratings

In [ ]:
df_hotels_filtered = df_hotels_filtered.set_index("Ratings")
df_hotels_filtered = df_hotels_filtered.drop(0.0, axis=0) # Delete all rows with Ratings 0
df_hotels_filtered

In [ ]:

#Sorting the Ratings
df_hotels_filtered.sort_values('Ratings', ascending=False, inplace=True) 

#Make new dataframe for top5 hotels
df_top5_hotels = df_hotels_filtered.head(5)
df_top5_hotels

Show top 5 hotels in Miami on map:

In [ ]:
# create map of Miami using latitude and longitude values
top5_miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_top5_hotels['lat'], df_top5_hotels['lng'], df_top5_hotels['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(top5_miami)  
    
top5_miami

3. Search for Nearby Places

First, we have created blank Dictionary 'd' to store multiple dataframes, Each Dataframe contains 10 Nearby Places under the radius of 500 Meters from the Hotel.
Then, using For Loop we can iterate to Every Hotel's Co-ordinates and search for Nearby Places. then Finally filtering Columns we will store Dataframe in Dictionary¶

In [ ]:

d = {}
for i in range(len(df_top5_hotels)):
    d[i] = pd.DataFrame()
    lat = df_top5_hotels['lat'].iloc[i]
    lng = df_top5_hotels['lng'].iloc[i]
    LIMIT=10
    radius = 500
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items) # flatten JSON
      
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    df = dataframe.loc[:, filtered_columns]
    
    # filter the category for each row
    df['venue.categories'] = df.apply(get_category_type, axis=1)
    
    # clean columns
    df.columns = [col.split('.')[-1] for col in df.columns]
    
    d[i] = df

In [ ]:

#Extracting Dataframes from 'd' and storing into new dataframe for each hotels
df_hotel1 = d[0]
df_hotel2 = d[1]
df_hotel3 = d[2]
df_hotel4 = d[3]
df_hotel5 = d[4]
df_hotel1

In [ ]:
# let's clean unwanted columns:

df_hotel1.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel2.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel3.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel4.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel5.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)

df_hotel5.head()

In [ ]:
# We have to Remove Special Characters from dataframe inorder to use Place names as a label on map

#replace (') with blank 
df_hotel1 = df_hotel1.replace('\'','',regex=True)
df_hotel2 = df_hotel2.replace('\'','',regex=True)
df_hotel3 = df_hotel3.replace('\'','',regex=True)
df_hotel4 = df_hotel4.replace('\'','',regex=True)
df_hotel5 = df_hotel5.replace('\'','',regex=True)

df_hotel5.head()

In [ ]:
# In order to have a better look at the center of interest, I add 0.04 to the longitud
miami_hotels_map = folium.Map(location=[latitude, longitude+0.04], zoom_start=14) # generate map centred around Miami

# add a marker to represent Hotel1
folium.Marker(
    [df_top5_hotels['lat'].iloc[0], df_top5_hotels['lng'].iloc[0]],
    popup=df_top5_hotels['name'].iloc[0],
    ).add_to(miami_hotels_map)

# add the Nearby Places as blue circle markers
for lat, lng, label in zip(df_hotel1['lat'], df_hotel1['lng'], df_hotel1['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel2
folium.Marker(
    [df_top5_hotels['lat'].iloc[1], df_top5_hotels['lng'].iloc[1]],
    popup=df_top5_hotels['name'].iloc[1],
    ).add_to(miami_hotels_map)

# add the Nearby Places as green circle markers
for lat1, lng1, label1 in zip(df_hotel2['lat'], df_hotel2['lng'], df_hotel2['name']):
    folium.features.CircleMarker(
        [lat1, lng1],
        radius=5,
        color='green',
        popup=label1,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel3
folium.Marker(
    [df_top5_hotels['lat'].iloc[2], df_top5_hotels['lng'].iloc[2]],
    popup=df_top5_hotels['name'].iloc[2],
    ).add_to(miami_hotels_map)

# add the Nearby Places as orange circle markers
for lat, lng, label in zip(df_hotel3['lat'], df_hotel3['lng'], df_hotel3['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel4
folium.Marker(
    [df_top5_hotels['lat'].iloc[3], df_top5_hotels['lng'].iloc[3]],
    popup=df_top5_hotels['name'].iloc[3],
    ).add_to(miami_hotels_map)

# add the Nearby Places as purple circle markers
for lat, lng, label in zip(df_hotel4['lat'], df_hotel4['lng'], df_hotel4['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel5
folium.Marker(
    [df_top5_hotels['lat'].iloc[4], df_top5_hotels['lng'].iloc[4]],
    popup=df_top5_hotels['name'].iloc[4],
    ).add_to(miami_hotels_map)

# add the Nearby Places as red circle markers
for lat, lng, label in zip(df_hotel5['lat'], df_hotel5['lng'], df_hotel5['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)

# display map
miami_hotels_map

4. Clustering Hotels with Nearby Places using K-Means

In [ ]:

#Combine all dataframe and store it in df_nearbyplaces then reset the index values
df_nearbyplaces = pd.concat([df_hotel1, df_hotel2, df_hotel3, df_hotel4, df_hotel5], axis=0).reset_index(drop=True)
df_nearbyplaces.head()

In [ ]:

number_of_clusters = 5 #define number of cluster to be generated. In our case it is 5 because we have 5 hotels

loc = df_nearbyplaces[['lat', 'lng']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(loc)

labels = kmeans.labels_

In [ ]:

df_nearbyplaces["Labels"] = labels #adding cluster labels to main dataframe
df_nearbyplaces.head()

In [ ]:
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
df_nearbyplaces['Labels'].unique()

visualize

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude+0.04], zoom_start=14) # generate map centred around Miami

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add Cluster circles to the map
for lon, lat in zip(cluster_centers[0], cluster_centers[1]):
    folium.CircleMarker([lon, lat], radius=50, color='yellow', fill=True, fill_opacity=0.25).add_to(map_clusters)
    folium.Marker(
    [lon, lat],
    popup='Cluster Center',
    ).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_nearbyplaces['lat'], df_nearbyplaces['lng'], df_nearbyplaces['name'], df_nearbyplaces['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Results:

This analysis Shows that it is possible to get information about hotels in Miami, and retriving ratings not of all of them. Some have no rating at all. All hotels were sorted based on ratings and Top 5 hotels were selected to recommend them to the user. Nearby places from selected 5 hotels were searched. From this data we can see that we have various categories of places that users can choose hotel based on having their interested nearby places to visit and enjoy. In the last step, clusters of best hotels and their surroundings were made for the costumer's convenience

Discussion:

People choose hotels also by their price per night and by their facilities. Moreover, the proximity to central bus/train stations can also be an issue plus many more features. Here there is a basic analysis. A deeper analysis can and must be made in order to get the best costumer satisfaction. For example, a list of restaurants of the costumer choice can be shown, museums, stores and more...

Conclusion:

In this project a recommendation of the Best Hotels in a City together with nearby places was achived. The aim was to help Tourists in narrowing down the search for optimal location for stay. By getting ratings of All Hotels from Foursquare data, Top 5 hotels were sorted and a collection of nearby places which satisfy some basic requirements regarding favourite Places was added. Clustering of those nearby places was then performed in order to create major zones of interest (containing nearby places) and addresses of those zone centers were created to be used as starting points for exploration by the costumers.
The final decission will be made by the costumers based on specific Requirements in every recommended zone. the costumer will choose and select the hotel to stay in by his own needs. This project can provide the costumer more than enough information in order to help him choose the best possible hotel in every city.